# Анализ базы данных StackOverflow

**Цель исследования:** Проанализировать базу данных StackOverflow — сервиса вопросов и ответов о программировании, где пользователи задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам. Проведение данного исследования позволит получить полезные инсайты о платформе StackOverflow и ее пользовательской базе, что может быть ценным для разработчиков, исследователей и специалистов в области программирования и IT-индустрии.

**Задачи:** 
1. Определить количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».
2. Определить, сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно.
3. Определить, сколько пользователей получили значки сразу в день регистрации.
4. Определить, сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос.
5. Пронумеровать записи таблицы vote_types в обратном порядке.
6. Отобрать 10 пользователей, которые поставили больше всего голосов типа Close.
7. Отобрать 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
8. Определить, сколько в среднем очков получает пост каждого пользователя.
9. Отобразить заголовки постов, которые были написаны пользователями, получившими более 1000 значков.
10. Выгрузить данные о пользователях из США и разделить пользователей на группы.
11. Отобразить лидеров каждой группы.
12. Посчитать ежедневный прирост новых пользователей в ноябре 2008 года.
13. Найти интервал между регистрацией и временем создания первого поста.
14. Вывести общую сумму просмотров постов за каждый месяц 2008 года.
15. Определить самых активных пользователей.
16. Вывести количество постов за 2008 год по месяцам.
17. Вывести сводные данные о постах.
18. Определить, сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой.
19. Определить, на сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года.
20. Выгрузить данные активности пользователя, который опубликовал больше всего постов за всё время.

**Описание данных:**

**Таблица `stackoverflow.badges`**

Хранит информацию о значках, которые присуждаются за разные достижения:

- `id` — Идентификатор значка, первичный ключ таблицы;
- `name` — Название значка;
- `user_id` — Идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users;
- `creation_date` — Дата присвоения значка.

**Таблица `stackoverflow.post_types`**

Содержит информацию о типе постов:

- `id` — Идентификатор типа поста, первичный ключ таблицы;
- `type` — Тип поста.

**Таблица `stackoverflow.posts`**

Содержит информацию о постах:

- `id` — Идентификатор поста, первичный ключ таблицы;
- `title` — Заголовок поста;
- `creation_date` — Дата создания поста;
- `favorites_count` — Число, которое показывает, сколько раз пост добавили в «Закладки»;
- `last_activity_date` — Дата последнего действия в посте, например комментария;
- `last_edit_date` — Дата последнего изменения поста;
- `user_id` — Идентификатор пользователя, который создал пост, внешний ключ к таблице users;
- `parent_id` — Если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом;
- `post_type_id` — Идентификатор типа поста, внешний ключ к таблице post_types;
- `score` — Количество очков, которое набрал пост;
- `views_count` — Количество просмотров.

**Таблица `stackoverflow.users`**

Содержит информацию о пользователях:

- `id` — Идентификатор пользователя, первичный ключ таблицы;
- `creation_date` — Дата регистрации пользователя;
- `display_name` — Имя пользователя;
- `last_access_date` — Дата последнего входа;
- `location` — Местоположение;
- `reputation` — Очки репутации, которые получают за хорошие вопросы и полезные ответы;
- `views` — Число просмотров профиля пользователя.

**Таблица `stackoverflow.vote_types`**

Содержит информацию о типах голосов (голос — это метка, которую пользователи ставят посту):

- `id` — Идентификатор типа голоса, первичный ключ;
- `name` — Название метки.

**Таблица `stackoverflow.votes`**

Содержит информацию о голосах за посты:

- `id` — Идентификатор голоса, первичный ключ;
- `post_id` — Идентификатор поста, внешний ключ к таблице posts;
- `user_id` — Идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users;
- `bounty_amount` — Сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту;
- `vote_type_id` — Идентификатор типа голоса, внешний ключ к таблице vote_types;
- `creation_date` — Дата назначения голоса.

#### 1. Определить количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(p.id)
FROM stackoverflow.posts p
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
WHERE pt.type = 'Question'
AND (p.score > 300
     OR p.favorites_count >= 100)

#### 2. Определить, сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно.

In [ ]:
SELECT ROUND(AVG(q_cnt))
FROM
    (SELECT p.creation_date::date AS dt,
           COUNT(p.id) AS q_cnt
    FROM stackoverflow.posts p
    JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
    WHERE pt.type = 'Question'
    AND p.creation_date BETWEEN '2008-11-01' AND '2008-11-19'
    GROUP BY dt) q

#### 3. Определить, сколько пользователей получили значки сразу в день регистрации.

In [ ]:
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE b.creation_date::date = u.creation_date::date

#### 4. Определить количество уникальных постов пользователя с именем Joel Coehoorn, которые получили хотя бы один голос.

In [ ]:
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON u.id = p.user_id
JOIN stackoverflow.votes v ON v.post_id = p.id
WHERE u.display_name = 'Joel Coehoorn'

#### 5. Пронумеровать записи таблицы vote_types в обратном порядке.
Выгрузить все поля таблицы vote_types. Добавить к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица отсортировать по полю id.

In [ ]:
SELECT *,
       ROW_NUMBER() OVER (ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id

#### 6. Отобрать 10 пользователей, которые поставили больше всего голосов типа Close. 
Отобразить таблицу из двух полей: идентификатор пользователя и количество голосов. Отсортировать данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT u.id,
       COUNT(v.id) AS vote_cnt
FROM stackoverflow.users u
JOIN stackoverflow.votes v ON v.user_id = u.id
JOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY u.id
ORDER BY vote_cnt DESC
LIMIT 10

#### 7. Отобрать 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразить несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвоить одно и то же место в рейтинге.
Отсортировать записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT u.id,
       COUNT(b.id) AS b_cnt,
       DENSE_RANK() OVER(ORDER BY COUNT(b.id) DESC) AS rank
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE b.creation_date BETWEEN '2008-11-15' AND '2008-12-16'
GROUP BY u.id
ORDER BY b_cnt DESC,
         u.id
LIMIT 10

#### 8. Определить, сколько в среднем очков получает пост каждого пользователя.
Сформировать таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывать посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM stackoverflow.posts
WHERE title IS NOT NULL
AND score <> 0

#### 9. Отобразить заголовки постов, которые были написаны пользователями, получившими более 1000 значков. 
Посты без заголовков не должны попасть в список.

In [ ]:
SELECT title
FROM
    (SELECT p.title AS title,
           COUNT(b.id) AS b_cnt
    FROM stackoverflow.posts p
    RIGHT JOIN stackoverflow.badges b ON p.user_id = b.user_id
    WHERE p.title IS NOT NULL
    GROUP BY p.title) c
WHERE b_cnt > 1000

#### 10. Выгрузить данные о пользователях из США и разделить пользователей на группы.
Разделить пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвоить группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразить в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны попасть в итоговую таблицу.

In [ ]:
SELECT u.id,
       u.views,
       CASE
           WHEN views>=350 THEN 1
           WHEN views>=100 THEN 2
           ELSE 3
       END AS view_group
FROM stackoverflow.users u
WHERE u.location LIKE '%United States%'
AND u.views <> 0

#### 11. Отобразить лидеров каждой группы.
Дополнить предыдущий запрос. Отобразить лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Вывести поля с идентификатором пользователя, группой и количеством просмотров. Отсортировать таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH groups AS
    (SELECT u.id,
           u.views,
           CASE
               WHEN u.views>=350 THEN 1
               WHEN u.views>=100 THEN 2
               ELSE 3
           END AS view_group
    FROM stackoverflow.users u
    WHERE u.location LIKE '%United States%'
    AND u.views <> 0)
SELECT id,
       view_group,
       views
FROM groups
WHERE views IN
    (SELECT MAX(views)
    FROM groups
    GROUP BY view_group)
ORDER BY views DESC,
         id

#### 12. Посчитать ежедневный прирост новых пользователей в ноябре 2008 года. 
Сформировать таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
WITH t AS
    (SELECT EXTRACT(DAY FROM creation_date) AS day_number,
           COUNT(id) AS users_cnt
    FROM stackoverflow.users
    WHERE creation_date BETWEEN '2008-11-01' AND '2008-12-01'
    GROUP BY day_number
    ORDER BY day_number)
SELECT *,
       SUM(users_cnt) OVER (ORDER BY day_number) AS users_sum
FROM t

#### 13. Найти интервал между регистрацией и временем создания первого поста.
Найти интервал между регистрацией и временем создания первого поста для каждого пользователя, который написал хотя бы один пост.
Отобразить:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
SELECT DISTINCT
       p.user_id,
       MIN(p.creation_date) OVER (PARTITION BY p.user_id) - u.creation_date AS time_diff
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON p.user_id = u.id

#### 14. Вывести общую сумму просмотров постов за каждый месяц 2008 года. 
Если данных за какой-либо месяц в базе нет, такой месяц следует пропустить. Результат отсортировать по убыванию общего количества просмотров.

In [ ]:
SELECT DATE_TRUNC('month', creation_date)::date AS mnth,
       SUM(views_count) AS views
FROM stackoverflow.posts
GROUP BY mnth
ORDER BY views DESC

#### 15. Определить самых активных пользователей.
Вывести имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывать. Для каждого имени пользователя вывести количество уникальных значений user_id. Отсортировать результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
       count(DISTINCT p.user_id)
FROM stackoverflow.posts p
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
JOIN stackoverflow.users u ON p.user_id = u.id
WHERE pt.type = 'Answer'
  AND p.creation_date::date <= u.creation_date::date + INTERVAL '1 month'
GROUP BY u.display_name
HAVING COUNT(p.post_type_id) > 100
ORDER BY u.display_name

#### 16. Вывести количество постов за 2008 год по месяцам. 
Отобрать посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортировать таблицу по значению месяца по убыванию.

In [ ]:
SELECT DATE_TRUNC('month', p.creation_date)::date AS dt,
       COUNT(p.id)
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON p.user_id = u.id
WHERE u.creation_date BETWEEN '2008-09-01' AND '2008-10-01'
  AND u.id IN
    (SELECT DISTINCT user_id
     FROM stackoverflow.posts
     WHERE creation_date BETWEEN '2008-12-01' AND '2009-01-01' )
GROUP BY dt
ORDER BY dt DESC

#### 17. Вывести сводные данные о постах.
Используя данные о постах, вывести несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id

#### 18. Определить, сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой. 
Для каждого пользователя отобрать дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число.

In [ ]:
WITH t AS
(SELECT user_id,
       COUNT(DISTINCT creation_date::date) AS days_cnt
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-12-01' AND '2008-12-08'
GROUP BY user_id)

SELECT ROUND(AVG(days_cnt))
FROM t

#### 19. Определить, на сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года.
Отобразить таблицу со следующими полями:
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлить значение процента до двух знаков после запятой.

In [ ]:
WITH t AS
(SELECT EXTRACT(MONTH FROM creation_date) AS month_num,
       COUNT(id) AS p_cnt
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-09-01' AND '2009-01-01'
GROUP BY month_num)

SELECT *,
       ROUND((p_cnt::numeric / LAG(p_cnt) OVER () * 100) - 100, 2) AS prcnt
FROM t

#### 20. Выгрузить данные активности пользователя, который опубликовал больше всего постов за всё время. 
Вывести данные за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
SELECT DISTINCT
       EXTRACT(WEEK FROM creation_date::date) AS week_numb,
       MAX(creation_date) OVER (PARTITION BY EXTRACT(WEEK FROM creation_date::date))
FROM stackoverflow.posts
WHERE user_id IN
    (SELECT user_id
    FROM stackoverflow.posts
    GROUP BY user_id
    ORDER BY COUNT(id) DESC
    LIMIT 1) AND
    creation_date BETWEEN '2008-10-01' AND '2008-11-01'

## Вывод

Анализ базы данных StackOverflow был проведен с использованием продвинутых средств SQL, таких как CASE, DISTINCT, ORDER BY, HAVING, JOIN, оконные функции, а также функции ранжирования и смещения. Эти инструменты позволили выполнить различные запросы и операции, такие как ранжирование результатов, фильтрация данных, сортировка, объединение таблиц и агрегирование информации, для получения нужных результатов и извлечения ценных показателей из базы данных.